In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns  
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import accuracy_score
from sklearn.tree import plot_tree
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,BaggingClassifier,AdaBoostClassifier,StackingClassifier,VotingClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import f1_score,accuracy_score,confusion_matrix
from scipy import stats
import xgboost as xgb
import warnings
warnings.filterwarnings("ignore")

#lettura database
data=pd.read_csv("credit_risk_dataset.csv")
print('shape originale', data.shape) 
print(data.info()) 
data.describe()


In [ ]:

#Procedo a cambiare il nome alle colonne per non doverle tradurle in testa ogni talvolta le leggo.
colonne_originali = data.columns
nuove_colonne = ['Età', 'Reddito', 'Diritto Abitazione', 'Tempo Attuale Impiego', 'Motivazione credito', 'Punteggio Credito', 'Ammontare del Credito', 'Interesse sul Credito', 'Status', 'Percentuale Credito', 'Cronologia Status', 'Cronologia Crediti']
colonne = dict(zip(colonne_originali, nuove_colonne)) 
data.rename(columns=colonne, inplace=True) 
print(colonne)

In [ ]:
#Ogni banca può applicare l'interesse che vuole, quindi non ha senso avere questo dato.
data.drop(['Interesse sul Credito'],axis=1,inplace=True)

#togliamo le righe duplicate, cosi evitiamo che vadano a finire sia all'interno del train che all'interno del test.
print(f'Ho {data.duplicated().value_counts()[True]} dati duplicati') 
data.drop_duplicates(inplace=True)
print('Dimensione del database una volte tolte le righe duplicate',data.shape) 

In [ ]:
print(f'Percento di data null: \n {data.isna().sum()/data.shape[0]*100}')

In [ ]:
# Analisi della variabile 'Tempo Attuale Impiego' 
data['Tempo Attuale Impiego'].plot.hist(bins=40)
plt.axvline(data['Tempo Attuale Impiego'].mode()[0], c='red')
plt.axvline(data['Tempo Attuale Impiego'].mean(), c='b')
plt.axvline(data['Tempo Attuale Impiego'].median(), c='y');
plt.axvline(data['Tempo Attuale Impiego'].quantile(.75), c='black', linestyle='--');

#Riempo i valori null
data['Tempo Attuale Impiego'].fillna(data['Tempo Attuale Impiego'].mode()[0], inplace=True)

#faccio un test per vedere che effettivamente sono stati eliminati
print(f'Percento di data null: \n {data.isna().sum()/data.shape[0]*100}')

In [ ]:
#Alcune finanziarie e banche valutano richieste di finanziamento purché il cliente, alla fine del finanziamento, non deve superare i 75 anni di età
mask_età_abile_per_crediti = (data.Età >= 18) & (data.Età <= 75) 

#L'aspettativa di vita in USA è in caduta libera da circa un decennio: è arrivata a 76,1 anni nel 2021
mask_tempo_utile_lavorativo = (data['Tempo Attuale Impiego'] <= (76.1 - 18))  

data = data[mask_età_abile_per_crediti & mask_tempo_utile_lavorativo]
print('Dimensione del database tenendo in conto l\'età anagrafica del cliente',data.shape)  


In [ ]:
#data[df['Reddito'] < 3.5e5].person_income.plot.hist(bins=40)
print(data['Reddito'].describe())
data[data['Reddito'] < 3.5e5]['Reddito'].plot.hist(bins=40)
plt.title('Distribuzione reddito.')

#Procedo quindi togliere gli outlier
data = data[data['Reddito'] < 3.5e5]

In [ ]:
ccol=data.select_dtypes(include=["object"]).columns
ncol=data.select_dtypes(include=["int","float"]).columns
print(ccol)


In [ ]:
num_cols = pd.DataFrame(data[ncol])  
# Cancello la colonna Status prima della visualizzazione
num_cols_hist = num_cols.drop(['Status'], axis=1)
# Visulizziamo la distribuzione per ogni variabile numerica
plt.figure(figsize=(12,16))

for i, col in enumerate(num_cols_hist.columns):
    idx = int('42'+ str(i+1))
    plt.subplot(idx)
    sns.histplot(num_cols_hist[col], color='deeppink', 
                 kde_kws={'color': 'forest green', 'lw': 2, 'label': 'KDE'}) # KDE: the kernel density estimate
    plt.title('Distribuzione del dato: ' +col, fontsize=14)
    plt.ylabel('Frequency', fontsize=12)
    plt.xlabel(col, fontsize=12)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.legend(['KDE'], prop={"size":12})

plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=0.95, hspace=0.35,
                    wspace=0.35)
plt.show()

In [ ]:
sns.pairplot(data=data, hue='Status') 

In [ ]:

print("The number of Categorical columns are:",len(ccol))
print("The number of Numerical columns are:",len(ncol)) 

for colonna in ccol:
    print(f'\n {colonna}: \n {data[colonna].value_counts(normalize=True).sort_index()}')

In [ ]:
data['Status'].value_counts(normalize=True) * 100

In [ ]:

X_num = StandardScaler().fit_transform(data[num_cols_hist.columns].values)
ohe = OneHotEncoder(sparse=False, drop='if_binary')
X_cat = ohe.fit_transform(data[ccol].values)
X = np.concatenate([X_num, X_cat], axis=1) 

y = data['Status']

feature_names = num_cols_hist.columns.tolist() + ohe.get_feature_names_out(ccol).tolist() 

print(X.shape, y.shape)
print(feature_names)


In [ ]:
num_cols_hist.columns 
df = pd.DataFrame(X_num, columns = num_cols_hist.columns)
# Visulizziamo la distribuzione per ogni variabile numerica, dopo averle scalato.
plt.figure(figsize=(12,16))

for i, col in enumerate(df.columns):
    idx = int('42'+ str(i+1))
    plt.subplot(idx)
    sns.histplot(df[col], color='deeppink', 
                 kde_kws={'color': 'forest green', 'lw': 2, 'label': 'KDE'}) # KDE: the kernel density estimate
    plt.title('Distribuzione del dato: ' +col, fontsize=14)
    plt.ylabel('Frequency', fontsize=12)
    plt.xlabel(col, fontsize=12)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.legend(['KDE'], prop={"size":12})

plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=0.95, hspace=0.35,
                    wspace=0.35)
plt.show()

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42) 
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
#controllo che abbia manenuto le proporzioni,nel target, dopo il shuffle/split
print(y.value_counts(normalize=True) * 100, y_test.value_counts(normalize=True) * 100 , y_train.value_counts(normalize=True) * 100)

In [ ]:

clf = DecisionTreeClassifier(criterion='entropy')
clf = clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
f, ax = plt.subplots(1,1, figsize=(20, 10))

plot_tree(clf, ax=ax, fontsize=10, filled=True, proportion=True, max_depth=2)

plt.show()

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=10)
neigh.fit(X_train, y_train)
y_pred=neigh.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
dt = RandomForestClassifier(n_estimators=1000,max_depth=100, n_jobs=11)
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
print(confusion_matrix(y_test,y_pred)) 
plt.figure(figsize=(7,7))
sns.heatmap(data=confusion_matrix(y_test,y_pred),annot=True)

In [ ]:
modelli = [
DecisionTreeClassifier(criterion='entropy'),
KNeighborsClassifier(n_neighbors=10),
RandomForestClassifier(n_estimators=1000,max_depth=100, n_jobs=11)
]



entries = []
for modello in modelli:
    nome_modello = modello.__class__.__name__
    clsffit = modello.fit(X_train,y_train)
    print('\t\tCLASSIFICATIION METRICS "{}":\n'.format(nome_modello))
    print(classification_report(y_test, modello.predict(X_test)))
